In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Set the random seed for reproducibility
np.random.seed(42)

# Define parameters for the damped harmonic oscillator
A = 1.0    # Amplitude
b = 0.05   # Damping coefficient
omega = 2 * np.pi / 5  # Angular frequency
T = 100    # Total time
dt = 0.1   # Time step

# Generate time points
t = np.arange(0, T, dt)

# Generate the damped oscillation data
x = A * np.exp(-b * t) * np.cos(omega * t)

# Add some noise to the data
noise = np.random.normal(scale=0.1, size=len(t))
x_noisy = x + noise

# Create a DataFrame
df = pd.DataFrame({'Time': t, 'Position': x_noisy})

df.to_excel('damped_oscillator.xlsx','series')

In [ ]:
series = pd.read_excel('damped_oscillator.xlsx',
                    sheet_name='series', header=0, index_col=0,
                     dtype=float)
print(series)

In [ ]:
# Plot the data
plt.figure(figsize=(10, 6))
plt.plot(series['Time'], series['Position'], label='Damped Oscillator with Noise')
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Damped Harmonic Oscillator')
plt.legend()
plt.show()

In [ ]:
# Define the SARIMAX model
#order = (2, 0, 2)  # (p, d, q) parameters for ARIMA
#seasonal_order = (1, 1, 1, 10)  # (P, D, Q, S) parameters for seasonal component

# find optimal parameters
import itertools

# Define the p, q parameters to take any value between 1 and 2, and the d parameter to take any value between 0 and 1
p = d = q = range(1, 3)
d = range(0, 2)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, d and q triplets (i.e., P, D, Q)
seasonal_pdq = [(x[0], x[1], x[2], 10) for x in list(itertools.product(p, d, q))]

In [ ]:
# Fit the model
#model = SARIMAX(df['Position'], order=pdq, seasonal_order=seasonal_pdq)
#results = model.fit()

import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

# Indentification of best model from different combinations of pdq and seasonal_pdq
best_score, best_param, best_paramSeasonal = float("inf"), None, None
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(series['Position'], order=param, seasonal_order=param_seasonal, enforce_invertibility=False)
            results = mod.fit(disp=False)
            if results.aic < best_score:
                best_score, best_param, best_paramSeasonal = results.aic, param, param_seasonal
            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue # if fit fails, just continue to the next parameters combionation

In [ ]:
# Print the best set of parameters according to the AIC.
print('The best model is ARIMA{}x{} - AIC:{}'.format(best_param, best_paramSeasonal, best_score))

In [ ]:
# Print the summary of the model
#order = (2, 0, 2)  # (p, d, q) parameters for ARIMA
#seasonal_order = (1, 1, 1, 10)  # (P, D, Q, S) parameters for seasonal component
mod = SARIMAX(series['Position'], order=best_param, seasonal_order=best_paramSeasonal, enforce_invertibility=False)
results=mod.fit()
print(results.summary())

In [ ]:
# Forecast future values for 50 steps
n_forecast = 50  # Number of steps to forecast
forecast = results.get_forecast(steps=n_forecast)
forecast_index = np.arange(T, T + n_forecast * dt, dt)

# Extract forecasted values and confidence intervals
forecast_values = forecast.predicted_mean
conf_int = forecast.conf_int()


In [ ]:
# Plot the results and the forecast
plt.figure(figsize=(10, 6))
predictions = results.predict()
plt.plot(series['Time'], series['Position'], label='Observed', color='blue')
plt.plot(series['Time'], predictions, label='Model', color='green')
plt.plot(forecast_index, forecast_values, label='Forecast', color='red')

plt.fill_between(forecast_index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Damped Harmonic Oscillator with SARIMAX Forecast')
plt.legend()
plt.show()

In [ ]:
# Calculate SST, SSR, SSE, and R2 
observed = series['Position']
mean_observed = np.mean(observed)
sst = np.sum((observed - mean_observed) ** 2)
sse = np.sum((observed - predictions) ** 2)
ssr = sst - sse
r2 = 1 - (sse / sst)

# Print the results
print(f'SST: {sst:.2f}')
print(f'SSE: {sse:.2f}')
print(f'SSR: {ssr:.2f}')
print(f'R²: {r2:.2f}')


In [ ]:
# Calculate degrees of freedom 
n = len(observed)  # Number of observations
p = len(results.params)  # Number of predictors in SARIMAX

# Calculate MST, MSE, MSR, and F-score  for Holt's linear method
mst = sst / (n - 1)
mse = sse / (n - p - 1)
msr = ssr / p
f_score = msr / mse

# Print the results
print(f'MST: {mst:.2f}')
print(f'MSE: {mse:.2f}')
print(f'MSR: {msr:.2f}')
print(f'F-score: {f_score:.2f}')

In [ ]:
# Now let's try with Holt's linear model
# Fit the Holt's linear trend model to same data
from statsmodels.tsa.api import Holt

model = Holt(df['Position'])
holt_results = model.fit(optimized=True)

# Make in-sample predictions
holt_predictions = holt_results.fittedvalues

# Make forecast for future points
forecast_steps = 50
holt_forecast = holt_results.forecast(steps=forecast_steps)
forecast_index = np.arange(T, T + n_forecast * dt, dt)

predictions = results.predict()
plt.figure(figsize=(10, 6))
plt.plot(df['Time'], df['Position'], label='Observed', color='blue')
plt.plot(df['Time'], holt_predictions, label='Predicted', color='red')
plt.plot(forecast_index, holt_forecast, label='Forecasted', color='pink')
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Damped Harmonic Oscillator: Observed vs Predicted')
plt.legend()
plt.show()